In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [4]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating a4e19c9..e8b5ed9
Fast-forward
 useEUCL.ipynb | 398 +++++++++++++++++++++++++++++++++++++++++++++-------------
 1 file changed, 309 insertions(+), 89 deletions(-)


In [5]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("evaluation.csv")

In [8]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install -q tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [9]:
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/2"

In [10]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [11]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [12]:
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder1 = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder1')
encoder2 = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder2')

FileNotFoundError: ignored

In [ ]:
def get_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    encoder_text_inputs = preprocessing_layer(text_input)
    
    outputs = encoder1(encoder_text_inputs)
    net = outputs['pooled_output']
    a = tfl.Dropout(0.3)(net)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dropout(rate=0.2)(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)

    reason_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='reason')
    encoder_reason_inputs = preprocessing_layer(reason_input)
    outputs = encoder2(encoder_reason_inputs)
    net = outputs['pooled_output']
    b = tfl.Dropout(0.3)(net)
    b = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(b)
    b = tfl.BatchNormalization()(b)
    b = tfl.Activation('relu')(b)

    distance = tfl.Lambda(euclidean_distance)([a, b])
    model = Model([text_input, reason_input], distance)
    return model

In [ ]:
model = get_model()

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 first (KerasLayer)             (None, 512)          256797824   ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 512)          0           ['first[0][0]']                  
                                                                                                  
 dense (Dense)                  (None, 64)           32832       ['dropout[0][0]']                
                                                                                              

In [ ]:
model.compile(optimizer="adam", loss = contrastive_loss, metrics = [tf.metrics.BinaryAccuracy()])

In [ ]:
loss_history = model.fit(siam_gen([df["text"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["text"], df_test["reason"]], 64),
                         steps_per_epoch=50, validation_steps=10, epochs = 50, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/50
50/50 [==============================] - 22s 211ms/step - loss: 0.5721 - binary_accuracy: 0.4934 - val_loss: 0.4786 - val_binary_accuracy: 0.5000
Epoch 2/50
50/50 [==============================] - 9s 186ms/step - loss: 0.4385 - binary_accuracy: 0.4469 - val_loss: 0.4294 - val_binary_accuracy: 0.4859
Epoch 3/50
50/50 [==============================] - 9s 186ms/step - loss: 0.3811 - binary_accuracy: 0.3147 - val_loss: 0.3519 - val_binary_accuracy: 0.2734
Epoch 4/50
50/50 [==============================] - 9s 188ms/step - loss: 0.3150 - binary_accuracy: 0.1937 - val_loss: 0.3126 - val_binary_accuracy: 0.2313
Epoch 5/50
50/50 [==============================] - 9s 187ms/step - loss: 0.2682 - binary_accuracy: 0.1331 - val_loss: 0.2973 - val_binary_accuracy: 0.2484
Epoch 6/50
22/50 [============>.................] - ETA: 4s - loss: 0.2473 - binary_accuracy: 0.1257

In [ ]:
y_preds = model.predict([df["text"], df["reason"]])
accuracy(df["label"],y_preds)

In [ ]:
y_preds = model.predict([df_test["text"], df_test["reason"]])
accuracy(df_test["label"],y_preds)

In [ ]:
model.save("models/useEUCL/")